Градиентный алгоритм

Нахождение точки минимума

In [46]:
import numpy as np
import matplotlib.pyplot as plt

def func(x):
    return 0.5 * x + 0.2 * x ** 2 - 0.1 * x ** 3

def df(x):
    return 0.5 + 0.4 * x - 0.3 * x ** 2

x = -4
n = 0.01
N = 200

for i in range(N):
    x -= n * df(x)
print(x)

-1.0758987284245605


Аппроксимация функции нахождением коэффициентов модели и нахождение среднего эмпирического риска по модели, входным и выходным данным самой функции и через градиентный алгоритм с производной функционала качества со сменой приближений

In [ ]:
import numpy as np

def func(x):
    return 0.1 * x**2 - np.sin(x) + 5.

def df(w, func_x, xx):
    return 2*((xx @ w - func_x) * xx).mean(1)

coord_x = np.arange(-5.0, 5.0, 0.1)
coord_y = func(coord_x)

eta = np.array([0.1, 0.01, 0.001, 0.0001])
w = np.array([0., 0., 0., 0.])
N = 200

coord_x = np.power(coord_x.reshape(-1, 1), range(4))
for i in range(N):
    w -= eta * df(w, coord_y, coord_x)

Q = np.square((coord_x @ w.T - coord_y)).mean()
print(w)
print(Q)

[ 4.98645739 -0.41780964  0.10273088  0.0317247 ]
0.13061463936073706


Отличие от предыдущего решения лишь в том, что дополнительно высчитывается начальное значение функционала качества, а также берется на каждом шаге одно случайное измерение из обучающей выборки, по которому определяются коэффициенты модели и пересчитывается функционал качества. Также выход из цикла возможен при достижении маленького значения функционала качества или к тому же по достижении определенного порога точности

In [ ]:
import numpy as np
def func(x):
    return 0.5 * x**2 - 0.1 * 1/np.exp(-x) + 0.5 * np.cos(2*x) - 2.

loss = lambda ax, fx: (ax-fx)**2
model_a = lambda xx, ww: xx @ ww

def df(w, func_x, xx):
    return 2*((xx @ w - func_x) * xx.T)

coord_x = np.arange(-5.0, 5.0, 0.1)
coord_y = func(coord_x)

sz = len(coord_x)
eta = np.array([0.01, 0.001, 0.0001, 0.01, 0.01])
w = np.array([0., 0., 0., 0., 0.])
N = 500 
lm = 0.02

np.random.seed(0)

coord_x = np.column_stack([
    coord_x**0,
    coord_x,
    coord_x**2,
    np.cos(2*coord_x),
    np.sin(2*coord_x)
])

Qe = loss(model_a(coord_x, w), coord_y).mean()

tolerance = 1e-6
prev_loss = Qe

for i in range(N):
    k = np.random.randint(0, sz)
    loss_k = loss(model_a(coord_x[k], w), coord_y[k])
    w -= eta * (df(w, coord_y[k], coord_x[k]))
    Qe = lm * loss_k + (1-lm) * Qe

    if abs(loss_k - prev_loss) < tolerance:
        break
    
    prev_loss = loss_k

Q = loss(model_a(coord_x, w), coord_y).mean()

Отличие от предыдущего решения лишь в том, что добавляется определенный параметр batch_size, который определяет размер батча, из которого высчитываются средние значения потерь и производной по среднему эмпирическому риску

In [5]:
import numpy as np
def func(x):
    return 0.5 * x + 0.2 * x ** 2 - 0.05 * x ** 3 + 0.2 * np.sin(4 * x) - 2.5

loss = lambda ax, fx: ((ax-fx)**2)
model_a = lambda xx, ww: xx @ ww

def df(w, func_x, xx):
    return 2*((xx @ w - func_x) * xx.T).mean(1)

coord_x = np.arange(-4.0, 6.0, 0.1)
coord_y = func(coord_x)

sz = len(coord_x)
eta = np.array([0.1, 0.01, 0.001, 0.0001])
w = np.array([0., 0., 0., 0.])
N = 500
lm = 0.02
batch_size = 50

coord_x = np.power(coord_x.reshape(-1, 1), range(4))

Qe = loss(model_a(coord_x, w), coord_y).mean()
np.random.seed(0)

tolerance = 1e-6
prev_loss = Qe

for i in range(N):
    k = np.random.randint(0, sz-batch_size)
    loss_k = (loss(model_a(coord_x[k:k+batch_size], w), coord_y[k:k+batch_size])).mean()
    w -= eta * (df(w, coord_y[k:k+batch_size], coord_x[k:k+batch_size]))
    Qe = lm * loss_k + (1-lm) * Qe

    if abs(loss_k - prev_loss) < tolerance:
        break
    
    prev_loss = loss_k

Q = loss(model_a(coord_x, w), coord_y).mean()

Две следующие программы являются повторениями двух предыдущих решений, но только это не задачи аппроксимации функций, а бинарной классификации

In [39]:
import numpy as np

def loss(w, x, y):
    M = np.dot(w, x) * y
    return np.log2(1 + np.exp(-M))

def df(w, x, y):
    M = np.dot(w, x) * y
    return -(np.exp(-M) * x * y) / (1 + np.exp(-M))

data_x = [(3.0, 4.9), (2.7, 3.9), (3.0, 5.5), (2.6, 4.0), (2.9, 4.3), (3.1, 5.1), (2.2, 4.5), (2.3, 3.3), (2.7, 5.1), (3.3, 5.7), (2.8, 5.1), (2.8, 4.9), (2.5, 4.5), (2.8, 4.7), (3.2, 4.7), (3.2, 5.7), (2.8, 6.1), (3.6, 6.1), (2.8, 4.8), (2.9, 4.5), (3.1, 4.9), (2.3, 4.4), (3.3, 6.0), (2.6, 5.6), (3.0, 4.4), (2.9, 4.7), (2.8, 4.0), (2.5, 5.8), (2.4, 3.3), (2.8, 6.7), (3.0, 5.1), (2.3, 4.0), (3.1, 5.5), (2.8, 4.8), (2.7, 5.1), (2.5, 4.0), (3.1, 4.4), (3.8, 6.7), (3.1, 5.6), (3.1, 4.7), (3.0, 5.8), (3.0, 5.2), (3.0, 4.5), (2.7, 4.9), (3.0, 6.6), (2.9, 4.6), (3.0, 4.6), (2.6, 3.5), (2.7, 5.1), (2.5, 5.0), (2.0, 3.5), (3.2, 5.9), (2.5, 5.0), (3.4, 5.6), (3.4, 4.5), (3.2, 5.3), (2.2, 4.0), (2.2, 5.0), (3.3, 4.7), (2.7, 4.1), (2.4, 3.7), (3.0, 4.2), (3.2, 6.0), (3.0, 4.2), (3.0, 4.5), (2.7, 4.2), (2.5, 3.0), (2.8, 4.6), (2.9, 4.2), (3.1, 5.4), (2.5, 4.9), (3.2, 5.1), (2.8, 4.5), (2.8, 5.6), (3.4, 5.4), (2.7, 3.9), (3.0, 6.1), (3.0, 5.8), (3.0, 4.1), (2.5, 3.9), (2.4, 3.8), (2.6, 4.4), (2.9, 3.6), (3.3, 5.7), (2.9, 5.6), (3.0, 5.2), (3.0, 4.8), (2.7, 5.3), (2.8, 4.1), (2.8, 5.6), (3.2, 4.5), (3.0, 5.9), (2.9, 4.3), (2.6, 6.9), (2.8, 5.1), (2.9, 6.3), (3.2, 4.8), (3.0, 5.5), (3.0, 5.0), (3.8, 6.4)]
data_y = [1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1]

x_train = np.array([[1, x[0], x[1]] for x in data_x])
y_train = np.array(data_y)

n_train = len(x_train)
w = [0.0, 0.0, 0.0]
nt = np.array([0.5, 0.01, 0.01])
lm = 0.01
N = 1000

Qe = 0
np.random.seed(0)

for i in range(N):
    k = np.random.randint(0, n_train-1)
    Qe = lm * loss(w, x_train[k], y_train[k]) + (1-lm) * Qe
    w -= nt * df(w, x_train[k], y_train[k])

Q = (x_train @ w * y_train < 0).mean()

In [14]:
import numpy as np

def loss(w, x, y):
    M = np.dot(x, w) * y
    return np.exp(-M).mean()

def df(w, x, y, size):
    M = np.dot(x, w) * y
    return -np.dot(x.T, np.exp(-M) * y) / size

data_x = [(5.8, 1.2), (5.6, 1.5), (6.5, 1.5), (6.1, 1.3), (6.4, 1.3), (7.7, 2.0), (6.0, 1.8), (5.6, 1.3), (6.0, 1.6), (5.8, 1.9), (5.7, 2.0), (6.3, 1.5), (6.2, 1.8), (7.7, 2.3), (5.8, 1.2), (6.3, 1.8), (6.0, 1.0), (6.2, 1.3), (5.7, 1.3), (6.3, 1.9), (6.7, 2.5), (5.5, 1.2), (4.9, 1.0), (6.1, 1.4), (6.0, 1.6), (7.2, 2.5), (7.3, 1.8), (6.6, 1.4), (5.6, 2.0), (5.5, 1.0), (6.4, 2.2), (5.6, 1.3), (6.6, 1.3), (6.9, 2.1), (6.8, 2.1), (5.7, 1.3), (7.0, 1.4), (6.1, 1.4), (6.1, 1.8), (6.7, 1.7), (6.0, 1.5), (6.5, 1.8), (6.4, 1.5), (6.9, 1.5), (5.6, 1.3), (6.7, 1.4), (5.8, 1.9), (6.3, 1.3), (6.7, 2.1), (6.2, 2.3), (6.3, 2.4), (6.7, 1.8), (6.4, 2.3), (6.2, 1.5), (6.1, 1.4), (7.1, 2.1), (5.7, 1.0), (6.8, 1.4), (6.8, 2.3), (5.1, 1.1), (4.9, 1.7), (5.9, 1.8), (7.4, 1.9), (6.5, 2.0), (6.7, 1.5), (6.5, 2.0), (5.8, 1.0), (6.4, 2.1), (7.6, 2.1), (5.8, 2.4), (7.7, 2.2), (6.3, 1.5), (5.0, 1.0), (6.3, 1.6), (7.7, 2.3), (6.4, 1.9), (6.5, 2.2), (5.7, 1.2), (6.9, 2.3), (5.7, 1.3), (6.1, 1.2), (5.4, 1.5), (5.2, 1.4), (6.7, 2.3), (7.9, 2.0), (5.6, 1.1), (7.2, 1.8), (5.5, 1.3), (7.2, 1.6), (6.3, 2.5), (6.3, 1.8), (6.7, 2.4), (5.0, 1.0), (6.4, 1.8), (6.9, 2.3), (5.5, 1.3), (5.5, 1.1), (5.9, 1.5), (6.0, 1.5), (5.9, 1.8)]
data_y = [-1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1]

x_train = np.array([[1, x[0], x[1]] for x in data_x])
y_train = np.array(data_y)

n_train = len(x_train)
w = np.array([0.0, 0.0, 0.0])
nt = np.array([0.5, 0.01, 0.01])
lm = 0.01
N = 500
batch_size = 10

Qe = 0
np.random.seed(0)

tolerance = 1e-6
prev_loss = Qe

for i in range(N):
    k = np.random.randint(0, n_train-batch_size-1)
    batch_x = x_train[k:k+batch_size]
    batch_y = y_train[k:k+batch_size]
    loss_k = loss(w, batch_x, batch_y)
    w -= nt * df(w, batch_x, batch_y, batch_size)
    Qe = lm * loss_k + (1-lm) * Qe

    if abs(loss_k - prev_loss) < tolerance:
        break

    prev_loss = loss_k

Q = (np.dot(x_train, w) * y_train < 0).mean()